In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from .code import inverse_problem as inv 

In [ ]:
path_to_experimental_data='./experimental_data/BURM7.EXP'
f=open(path_to_experimental_data)
path_to_AB_MN_data=('./experimental_data/'+f.readline()).strip()
f.close()
dat = np.genfromtxt(path_to_experimental_data, comments= '$' , skip_header = 3)
experimental_rhoa = dat.T[2] # измеренные кажущиеся сопротивления
stt=np.genfromtxt(path_to_AB_MN_data,comments= '$')
AB2 = stt.T[0] # Полуразнос

In [ ]:
N_layers_list=[2,3,4,5] # список из количества слоёв в моделях
exp_data=np.array([AB2, experimental_rhoa]).T # Данные
inverse=inv.aprox_inverse_problem(N_layers_list,exp_data) 
results=inverse[0] # модели полученные из обратной задачи 
ind=inverse[1] #  индекс модели с наименьшим loss_N_layers в results
print(ind)

In [ ]:
#Вывод количества слоёв в модели, её параметров и полученной ошибки
for i in range (len(N_layers_list)):
    print('\nnumber of layers= ', N_layers_list[i],' \nparam =',results[i].x,'\nloss =',results[i].fun)

In [ ]:
# Визуализация
for i in range(len(N_layers_list)):
    if N_layers_list[i]==ind:
        plt.scatter(AB2,inv.aprox_rhoa_N_layers(AB2, results[i].x), c='black',label=f'{N_layers_list[i]}_layers, best loss')
    else: plt.loglog(AB2,inv.aprox_rhoa_N_layers(AB2, results[i].x), label=f'{N_layers_list[i]}_layers')
plt.loglog(AB2,experimental_rhoa,c='red', label='experimental data')
plt.legend()